# **Créditos por porcentaje de UMAS**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "queretaro" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_queretaro',
 'df_financiamientos_2020_queretaro',
 'df_financiamientos_2021_queretaro',
 'df_financiamientos_2022_queretaro',
 'df_financiamientos_2023_queretaro',
 'df_financiamientos_2024_queretaro']

## 2022

In [2]:
finance = df_financiamientos_2022_queretaro[['id','ingresos_rango']]
registros_0 = finance[finance['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2022 = finance.shape
print(df_2022)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

40
(8873, 2)


In [3]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].astype(str)
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].replace({
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].replace({
    
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10416\2444930754.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2' '2' '3' ... '1' '0' '0']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].astype(str)


,id,ingresos_rango
0,1,2.61-4.00
1,2,2.61-4.00
2,3,4.01-6.00
3,4,2.61-4.00
4,5,4.01-6.00
...,...,...
8868,8869,9.01-12.00
8869,8870,0
8870,8871,≤ 2.60
8871,8872,0


In [4]:
# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['ingresos_rango'] != '0']
# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']
# Calcular los porcentajes
total_registros = len(finance)
conteo_ingresos['porcentajes'] = round((conteo_ingresos['conteo'] / total_registros) * 100, 2)
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,> 12.00,2309,26.14
1,6.01-9.00,1604,18.16
2,4.01-6.00,1339,15.16
3,9.01-12.00,1320,14.94
4,≤ 2.60,1195,13.53
5,2.61-4.00,1066,12.07


In [5]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
4,≤ 2.60,1195,13.53
5,2.61-4.00,1066,12.07
2,4.01-6.00,1339,15.16
1,6.01-9.00,1604,18.16
3,9.01-12.00,1320,14.94
0,> 12.00,2309,26.14


In [6]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2022)',
    #title_font_size = 22,
    xaxis=dict(title='UMAS',),
    yaxis=dict(
        title='% Créditos', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
guardar_grafico_como_html(fig, 'g_bar_creditos_umas_2022', carpeta='assets\graficas')
fig.show()

<>:32: SyntaxWarning: invalid escape sequence '\g'
<>:47: SyntaxWarning: invalid escape sequence '\g'
<>:32: SyntaxWarning: invalid escape sequence '\g'
<>:47: SyntaxWarning: invalid escape sequence '\g'
C:\Users\yoe11\AppData\Local\Temp\ipykernel_10416\3178355634.py:32: SyntaxWarning: invalid escape sequence '\g'
  def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
C:\Users\yoe11\AppData\Local\Temp\ipykernel_10416\3178355634.py:47: SyntaxWarning: invalid escape sequence '\g'
  guardar_grafico_como_html(fig, 'g_bar_creditos_umas_2022', carpeta='assets\graficas')


In [7]:
13.5+12.1+15.2+18.2+14.9+26.1

100.0

## 2023

In [8]:
finance = df_financiamientos_2023_queretaro[['id','ingresos_rango']]
registros_0 = finance[finance['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2022 = finance.shape
print(df_2022)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

71
(8364, 2)


In [9]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].astype(str)
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].replace({
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].replace({
    
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10416\2444930754.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2' '1' '2' ... '3' '4' '4']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



,id,ingresos_rango
0,1,2.61-4.00
1,2,≤ 2.60
2,3,2.61-4.00
3,4,6.01-9.00
4,5,2.61-4.00
...,...,...
8359,8360,≤ 2.60
8360,8361,2.61-4.00
8361,8362,4.01-6.00
8362,8363,6.01-9.00


In [10]:
# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['ingresos_rango'] != '0']
# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']
# Calcular los porcentajes
total_registros = len(finance)
conteo_ingresos['porcentajes'] = round((conteo_ingresos['conteo'] / total_registros) * 100, 2)
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,> 12.00,2170,26.17
1,6.01-9.00,1632,19.68
2,4.01-6.00,1362,16.42
3,9.01-12.00,1243,14.99
4,2.61-4.00,1057,12.75
5,≤ 2.60,829,10.00


In [11]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
5,≤ 2.60,829,10.00
4,2.61-4.00,1057,12.75
2,4.01-6.00,1362,16.42
1,6.01-9.00,1632,19.68
3,9.01-12.00,1243,14.99
0,> 12.00,2170,26.17


In [12]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2022)',
    #title_font_size = 22,
    xaxis=dict(title='UMAS',),
    yaxis=dict(
        title='% Créditos', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
guardar_grafico_como_html(fig, 'g_bar_creditos_umas_2023', carpeta='assets\graficas')
fig.show()

<>:32: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:32: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10416\2535799959.py:32: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10416\2535799959.py:47: SyntaxWarning:

invalid escape sequence '\g'



In [13]:
10+12.8+16.4+19.7+15+26.2

100.10000000000001

## 2024

In [13]:
finance = df_financiamientos_2024_queretaro[['id','ingresos_rango']]
registros_0 = finance[finance['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2022 = finance.shape
print(df_2022)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

131
(5435, 2)


In [14]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].astype(str)
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].replace({
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].replace({
    
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10416\2444930754.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '0' '0' ... '6' '3' '4']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



,id,ingresos_rango
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
5430,5431,> 12.00
5431,5432,6.01-9.00
5432,5433,> 12.00
5433,5434,4.01-6.00


In [15]:
# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['ingresos_rango'] != '0']
# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']
# Calcular los porcentajes
total_registros = len(finance)
#conteo_ingresos['porcentajes'] = (conteo_ingresos['conteo'] / total_registros) * 100
conteo_ingresos['porcentajes'] = round((conteo_ingresos['conteo'] / total_registros) * 100, 2)
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,> 12.00,1204,22.70
1,6.01-9.00,1027,19.36
2,4.01-6.00,933,17.59
3,9.01-12.00,804,15.16
4,2.61-4.00,779,14.69
5,≤ 2.60,557,10.50


In [16]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
5,≤ 2.60,557,10.50
4,2.61-4.00,779,14.69
2,4.01-6.00,933,17.59
1,6.01-9.00,1027,19.36
3,9.01-12.00,804,15.16
0,> 12.00,1204,22.70


In [17]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2022)',
    #title_font_size = 22,
    xaxis=dict(title='UMAS',),
    yaxis=dict(
        title='% Créditos', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
guardar_grafico_como_html(fig, 'g_bar_creditos_umas_2024', carpeta='assets\graficas')
fig.show()

<>:32: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:32: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10416\3917510289.py:32: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10416\3917510289.py:47: SyntaxWarning:

invalid escape sequence '\g'



In [19]:
10.5+14.7+17.6+19.4+15.2+22.7

100.1